<img src="images/header-transparent.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Máster</font></h2>

<h1><font color="#6B001F" size=5>SERENDIPITY: Servicio web para la recomendacIón de playlists a partir de otra playlist</font></h1>
<h2><font color="#92002A" size=3>Parte 4 - Infraestructura</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Máster Universitario en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
    * [1.1 - Infraestrucura como Código (IaC)](#section11)
    * [1.2 - Terraform](#section12)
* [2. Infraestructura para los procesos de aprendizaje automático](#section2)
* [3. Infraestructura para el servicio de recomendación](#section3)

<br>

---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Antes de comenzar a implementar nuestro servicio de recomendación de playlists, vamos a definir toda la infraestructura que vamos a necesitar desplegar en el proveedor de servicios cloud.

Se ha tomado la decisión de distinguir dos partes en la infraestructura que vamos a diseñar:
* Infraestructura para los procesos de aprendizaje automático / machine learning (servicios *Machine Learning as a Service* , *MLaaS*).
* Infraestructura para el servicio de recomendación (servicios *serverless*).


Para realizar el despliegue, lo haremos mediante *Infraestructura como Código* o *IaC* (*Infraestructure as Code*) empleando la herramienta *Terraform*.


<br>

---

<br>

<a id="section11"></a>
### <font color="#92002A">1.1 - Infraestructura como código (IaC)</font>
<br>

*Infraestructura como código* es la administración de la infraestructura (redes, máquinas virtuales, balanceadores de carga, topología de conexión, ...) en un modelo descriptivo, empleando el mismo control de versiones que utiliza el equipo de DevOps para el código fuente. Al igual que el principio de que el mismo código fuente genera el mismo código binario, un modelo de IaC genera el mismo entorno cada vez que se aplica. IaC es una práctica clave de DevOps y se usa junto con la entrega continua. [[Fuente]](https://docs.microsoft.com/es-es/devops/deliver/what-is-infrastructure-as-code)

<center><img src="images/iac.png" alt="IaC"></center>

<br>

---

<br>

<a id="section12"></a>
### <font color="#92002A">1.2 - Terraform</font>
<br>

Con todas las características que nos aporta la *Infraestructura como Código*, nos encontramos con los siguientes desafíos:

* Se necesita aprender a programar.
* Se desconoce el impacto del cambio.
* En ocasiones, es necesito revertir el cambio.
* No se pueden rastrear cambios.
* No se puede automatizar un recurso
* Múltiples entornos (diferentes proveedores) para la infraestructura.

Para solucionar estos inconvenientes, vamos a emplear la herramienta *Terraform*.


<br><center><img src="images/terraform-logo.png" width=512 alt="IaC"></center><br>


*[Terraform](https://www.terraform.io/)* es una herramienta para Infraestructura como Código desarrollada por *[HashiCorp](https://www.hashicorp.com/)*. Se utiliza para definir y aprovisionar una infraestructura completa utilizando el lenguaje declarativo [*HashiCorp Configuration Language* (*HCL*)](https://www.terraform.io/docs/language/syntax/configuration.html). Es muy similar a herramientas como *CloudFormation* de *Amazon Web Services* o *Azure Resource Manager* de *Microsoft*. 

Las principales ventajas de *Terraform* son las siguientes:
* Realiza orquestación, no sólo gestión de configuración.
* Admite múltiples proveedores como *AWS*, *Azure*, *GCP*, *DigitalOcean* y muchos más.
* Proporciona una infraestructura inmutable donde la configuración cambia sin problemas.
* Utiliza un lenguaje fácil de entender.
* Es fácil de portar a otro proveedor.
* Admite arquitectura de solo cliente, por lo que no es necesario administrar la configuración adicional en un servidor.

<br>

El ciclo de vida de *Terraform*, consiste en las fases: **init** (*inicializar*), **plan** (*planificar*), **apply** (*aplicar*) y **destroy** (*destruir*).

<br><center><img src="images/terraform-lifecycle.png" alt="terraform lifecycle"></center><br>

* *terraform init*: Inicializa el entorno Terraform de forma local. Por lo general, se ejecuta sólo una vez por sesión.
* *terraform plan*: Compara el estado de Terraform con el estado actual en la nube, creando y mostrando un plan de ejecución. Esta acción no cambia la implementación (solo lectura).
* *terraform apply*: Ejecuta el plan (actualiza la implementación en la nube). 
* *terraform destroy*: Elimina todos los recursos que se rigen por el entorno de Terraform.

[Fuente 1](https://k21academy.com/terraform-iac/terraform-beginners-guide/) [Fuente 2](https://comparacloud.com/servicios-clouds/terraform/)
<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<br>


<a id="section2"></a>
## <font color="#92002A">2 - Infraestructura para los procesos de aprendizaje automático</font>

<br>

La arquitectura de la primera infraestructura a definir pertenece a los procesos de aprendizaje automático. Esta infraestructura estará formada por instancias de servicios de *Machine Learning as a Service* disponibles en *Microsoft Azure*. En particular vamos a crear los siguientes recursos:
1) Área de trabajo de machine learning.
2) Instancia de proceso *Fsv2*, para realizar el entrenamiento del modelo.
3) Modelo
4) Punto de conexión (o endpoint), para consumir las predicciones realizadas por el modelo.

<br>

<div class="alert alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i>
__Nota__: El área de trabajo de machine learning requiere de los siguientes recursos para su creación:
    <lu>
        <li><i>Application Insights</i>: Recurso para obtener las métricas de los elementos que se encuentran bajo el área de trabajo de machine learning.</li>
        <li><i>Key Vault</i>: Almacen de claves criptográficas y otros secretos usados por las aplicaciones y los servicios de machine learning.</li>
        <li><i>Storage Account</i>: Cuenta de almacenamiento donde se guardarán los artefactos creados en el área de trabajo, tales como libretas, modelos, experimentos, etc ...</li>
    </lu>
</div>

<br>

Todos estos recursos estarán agrupados en un grupo de recursos (agrupación lógica de recursos en *Microsoft Azure*). A continuación, podemos ver un diagrama de los recursos que conforman la infraestructura para los procesos de aprendizaje automático:

<center><img src="images/AzureML.png" alt="Azure ML"></center>

A modo de resumen, dentro del área de trabajo de machine learning, se creará una instancia de proceso para entrenar el modelo de recomendación de nuestro sistema. Una vez entrenado y obtenido el modelo, lo almacenamos en la colección de modelos y creamos un punto de conexión para consumir las predicciones de dicho modelo.

Con la infraestructura ya definida, procedemos a desplegar los recursos mediante *Terraform*.

<br>

<div class="alert alert-info">

<i class="fa fa-info-circle" aria-hidden="true"></i>
__Nota__: Tanto el *modelo* como el *punto de acceso*, son elementos que serán creados posteriormente mediante el área de trabajo o de forma programática.
</div>

<br>

<font color="#92002A" size=3>Ficheros empleados para Terraform</font>
    

* **main.tf** : Fichero que contiene la implementación de la arquitectura en HCL.

```json
data "azurerm_client_config" "current" {}

resource "azurerm_resource_group" "rg" {
  name     = var.resource_group_name[terraform.workspace]
  location = var.resource_group_location
}

resource "azurerm_application_insights" "mlw_insights" {
  name                = var.mlw_application_insights_name
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name
  application_type    = "web"
}

resource "azurerm_key_vault" "mlw_key" {
  name                = var.mlw_key_vault_name
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name
  tenant_id           = data.azurerm_client_config.current.tenant_id
  sku_name            = "standard"
}

resource "azurerm_storage_account" "mlw-storage" {
  name                     = var.mlw_storage_account_name
  location                 = azurerm_resource_group.rg.location
  resource_group_name      = azurerm_resource_group.rg.name
  account_tier             = "Standard"
  account_replication_type = "LRS"
}

resource "azurerm_machine_learning_workspace" "mlw" {
  name                    = var.ml_workspace_name
  location                = azurerm_resource_group.rg.location
  resource_group_name     = azurerm_resource_group.rg.name
  application_insights_id = azurerm_application_insights.mlw_insights.id
  key_vault_id            = azurerm_key_vault.mlw_key.id
  storage_account_id      = azurerm_storage_account.mlw-storage.id

  identity {
    type = "SystemAssigned"
  }
}

resource "azurerm_virtual_network" "mlw-vnet" {
  name                = var.mlw_virtual_network_name
  address_space       = ["10.1.0.0/16"]
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name
}

resource "azurerm_subnet" "mlw-subnet" {
  name                 = var.mlw_subnetwork_name
  resource_group_name  = azurerm_resource_group.rg.name
  virtual_network_name = azurerm_virtual_network.mlw-vnet.name
  address_prefixes     = ["10.1.0.0/24"]
}

resource "azurerm_machine_learning_compute_instance" "computeinstance" {
  name                          = var.ml_compute_instance_name
  location                      = azurerm_resource_group.rg.location
  machine_learning_workspace_id = azurerm_machine_learning_workspace.mlw.id
  virtual_machine_size          = "Standard_F32s_v2"
  authorization_type            = "personal"
  ssh {
    public_key = var.ssh_key
  }
  subnet_resource_id = azurerm_subnet.mlw-subnet.id
  description        = "Main compute instance for Serenditipy-ML"
}
```

* **variables.tf** : Contiene la definición de las variables empleadas.
* **terraform.tfvars** : Contiene el valor de las variables definidas.
* **providers.tf** : Contiene la información del proveedor.
* **outputs.tf** : Contiene variables que resultan de interés y que se obtienen tras desplegarse la infraestructura en la nube.

<br>

Llegados a este punto, queda definida e desplegada la infraestructura para los servicios de machine learning de nuestro proyecto.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<br>

<a id="section3"></a>
## <font color="#92002A">3 - Infraestructura para el servicio de recomendación</font>

<br>

Llega el momento de crear la infraestructura que utilizara nuestro servicio de recomendación de playlists. Vamos a necesitar los siguientes recursos *serverless*:
1) *Azure Cosmos DB* (base de datos).
2) Aplicación de funciones, conocida como *Azure Functions*.
3) Servicio de *API Management*.
4) Aplicación web alojada en un contenedor *blob* dentro de nuestra cuenta de almacenamiento.

Todos estos recursos estarán bajo un único grupo de recursos, diferente del anterior:

<center><img src="images/AzureServerless.png" alt="Infraestructura Serverless"></center><br>

<br>

En la base de datos que crearemos en *Cosmos DB*, almacenaremos toda la información relativa a las pistas, artistas, álbumes, playlists, etc. Esta información será acesible mediante *Azure Functions*. Estas funciones también se encargarán de conectarse al punto de conexión del modelo de recomendación de playlist y también de realizar la autenticación en *Spotify*, para leer las playlist de un usuario. Mediante *API Managment* enlazaremos las funciones con el servicio API REST que crearemos, al igual que se atenderán las peticiones de autenticación de *Spotify*. Por último, tendremos almacenada una aplicación web en un *blob* (contenedor de archivos) que hará uso de la API.

<br>

<font color="#92002A" size=3>Ficheros empleados para Terraform</font>
    
<br>

Para terminar, vamos a crear la infraestructura mediante *Terraform*:

* **main.tf** : Fichero que contiene la implementación de la arquitectura en HCL.

```json
resource "azurerm_resource_group" "rg" {
  name     = var.resource_group_name[terraform.workspace]
  location = var.resource_group_location
}

resource "azurerm_storage_account" "storage" {
  name                     = var.rec_storage_account_name
  resource_group_name      = azurerm_resource_group.rg.name
  location                 = azurerm_resource_group.rg.location
  account_tier             = "Standard"
  account_replication_type = "LRS"
  account_kind             = "StorageV2"

  static_website {
    index_document = var.webapp_index_document
  }
}

resource "azurerm_cosmosdb_account" "cosmos_account" {
  name                = var.cosmosdb_account_name
  resource_group_name = azurerm_resource_group.rg.name
  location            = azurerm_resource_group.rg.location
  offer_type          = "Standard"
  kind                = "MongoDB"

  enable_automatic_failover = false

  capabilities {
    name = "EnableMongo"
  }

  capabilities {
    name = "MongoDBv3.4"
  }

  consistency_policy {
    consistency_level       = "BoundedStaleness"
    max_interval_in_seconds = 10
    max_staleness_prefix    = 200
  }

  geo_location {
    location          = azurerm_resource_group.rg.location
    failover_priority = 0
  }
}

resource "azurerm_cosmosdb_mongo_database" "database" {
  name                = var.cosmosdb_mongodb_db_name
  resource_group_name = azurerm_cosmosdb_account.cosmos_account.resource_group_name
  account_name        = azurerm_cosmosdb_account.cosmos_account.name
  throughput          = 400
}

resource "azurerm_app_service_plan" "appserviceplan" {
  name                = var.app_service_plan_name
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name

  sku {
    tier = "Standard"
    size = "F1"
  }
}

resource "azurerm_function_app" "functionapp" {
  name                       = var.function_app_name
  location                   = azurerm_resource_group.rg.location
  resource_group_name        = azurerm_resource_group.rg.name
  app_service_plan_id        = azurerm_app_service_plan.appserviceplan.id
  storage_account_name       = azurerm_storage_account.storage.name
  storage_account_access_key = azurerm_storage_account.storage.primary_access_key
}

resource "azurerm_api_management" "apimanagment" {
  name                = var.api_managment_name
  location            = azurerm_resource_group.rg.location
  resource_group_name = azurerm_resource_group.rg.name
  publisher_name      = var.publisher_name
  publisher_email     = var.publisher_email

  sku_name = "Consumption_0"
}
```

* **variables.tf** : Contiene la definición de las variables empleadas.
* **terraform.tfvars** : Contiene el valor de las variables definidas.
* **providers.tf** : Contiene la información del proveedor 

<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>